# Transform OpenAPI apis into MCP tools using Bedrock AgentCore Gateway

## Overview
Customers can bring OpenAPI spec in JSON or YAML and transform the apis into MCP tools using Bedrock AgentCore Gateway. We will demonstrate building a Mars Weather agent that calls NASA's Open APIs using API key. 

The Gateway workflow involves the following steps to connect your agents to external tools:
* **Create the tools for your Gateway** - Define your tools using schemas such as OpenAPI specifications for REST APIs. The OpenAPI specifications are then parsed by Amazon Bedrock AgentCore for creating the Gateway.
* **Create a Gateway endpoint** - Create the gateway that will serve as the MCP entry point with inbound authentication.
* **Add targets to your Gateway** - Configure the OpenAPI targets that define how the gateway routes requests to specific tools. All the APIs that part of OpenAPI file will become an MCP-compatible tool, and will be made available through your Gateway endpoint URL. Configure outbound authorization for each OpenAPI Gateway target. 
* **Update your agent code** - Connect your agent to the Gateway endpoint to access all configured tools through the unified MCP interface.

![How does it work](images/openapi-gateway-apikey.png)

### Tutorial Details


| Information          | Details                                                   |
|:---------------------|:----------------------------------------------------------|
| Tutorial type        | Interactive                                               |
| AgentCore components | AgentCore Gateway, AgentCore Identity                     |
| Agentic Framework    | Strands Agents                                            |
| Gateway Target type  | OpenAPI                                                   |
| Agent                | Mars Weather agent                                        |
| Inbound Auth IdP     | Amazon Cognito                                            |
| Outbound Auth        | API Key                                                   |
| LLM model            | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| Tutorial components  | Creating AgentCore Gateway and Invoking AgentCore Gateway |
| Tutorial vertical    | Cross-vertical                                            |
| Example complexity   | Easy                                                      |
| SDK used             | boto3                                                     |

In the first part of the tutorial we will create some AmazonCore Gateway targets

### Tutorial Architecture
In this tutorial we will transform operations defined in OpenAPI yaml/json file into MCP tools and host it in Bedrock AgentCore Gateway.
For demonstration purposes, we will build a Mars Weather agent that answers queries related to weather in Mars. The agent uses Open APIs of NASA. The solution uses Strands Agent using Amazon Bedrock models
In our example we will use a very simple agent with getInsightWeather tool for Mars weather.

## Prerequisites

To execute this tutorial you will need:
* Jupyter notebook (Python kernel)
* uv
* AWS credentials
* Amazon Cognito

In [1]:
# Make sure you download the latest botocore and boto3 libraries.
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

✅ 'uv' is already installed.
📦 Installing botocore, boto3 using uv...


Using Python 3.10.16 environment at: /Users/youngjik/Documents/GitHub/amazon-bedrock-agentcore-samples/.venv
Audited 2 packages in 26ms


In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
os.environ['AWS_ACCESS_KEY_ID'] = '' # set the access key
os.environ['AWS_SECRET_ACCESS_KEY'] = '' # set the secret 
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1' # set the AWS region

In [3]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

In [4]:
#### Create an IAM role for the Gateway to assume
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

Role already exists -- deleting and creating it again
policies: {'PolicyNames': ['AgentCorePolicy'], 'IsTruncated': False, 'ResponseMetadata': {'RequestId': '55dc624d-8a16-4a54-ba06-d8f7fa2bd063', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 19 Jul 2025 08:14:24 GMT', 'x-amzn-requestid': '55dc624d-8a16-4a54-ba06-d8f7fa2bd063', 'content-type': 'text/xml', 'content-length': '380'}, 'RetryAttempts': 0}}
deleting agentcore-sample-lambdagateway-role
recreating agentcore-sample-lambdagateway-role
attaching role policy agentcore-sample-lambdagateway-role
Agentcore gateway role ARN:  arn:aws:iam::720025708473:role/agentcore-sample-lambdagateway-role


# Create Amazon Cognito Pool for Inbound authorization to Gateway

In [5]:
# Creating Cognito User Pool 
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL  
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

Creating or retrieving Cognito resources...
Found domain for user pool us-east-1_1kHdoz8IG: us-east-11khdoz8ig (https://us-east-11khdoz8ig.auth.us-east-1.amazoncognito.com)
User Pool ID: us-east-1_1kHdoz8IG
Resource server ensured.
Client ID: nkd00psg2h021e3e1b7n8jttd
https://cognito-idp.us-east-1.amazonaws.com/us-east-1_1kHdoz8IG/.well-known/openid-configuration


# Create the Gateway 

In [6]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoGWOpenAPIAPIKeyNasaOAI',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with OpenAPI target'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

{'ResponseMetadata': {'RequestId': '7922efb4-1102-4d21-90f8-a668d9aa81dc', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sat, 19 Jul 2025 08:14:33 GMT', 'content-type': 'application/json', 'content-length': '1014', 'connection': 'keep-alive', 'x-amzn-requestid': '7922efb4-1102-4d21-90f8-a668d9aa81dc', 'x-amzn-remapped-x-amzn-requestid': '94c85c7a-0e50-44da-9a83-84fac59d904d', 'x-amzn-remapped-content-length': '1014', 'x-amzn-remapped-connection': 'keep-alive', 'x-amz-apigw-id': 'N8oMeGzroAMEJCA=', 'x-amzn-trace-id': 'Root=1-687b53e9-45abdd224519fc17146f2e40', 'x-amzn-remapped-date': 'Sat, 19 Jul 2025 08:14:33 GMT'}, 'RetryAttempts': 2}, 'gatewayArn': 'arn:aws:bedrock-agentcore:us-east-1:720025708473:gateway/demogwopenapiapikeynasaoai-vxgiwvbxp9', 'gatewayId': 'demogwopenapiapikeynasaoai-vxgiwvbxp9', 'gatewayUrl': 'https://demogwopenapiapikeynasaoai-vxgiwvbxp9.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp', 'createdAt': datetime.datetime(2025, 7, 19, 8, 14, 33, 194146, tzinfo=

# Transforming NASA Open APIs into MCP tools using Bedrock AgentCore Gateway

We are going to have a Mars Weather agent getting weather data from Nasa's Open APIs. You will need to register for Nasa Insight API [here](https://api.nasa.gov/). It's free! Once you register, you will get an API Key in your email. Use the API key to configure the credentials provider for creating the OpenAPI target.

In [7]:
import boto3
from pprint import pprint
from botocore.config import Config

acps = boto3.client(service_name="bedrock-agentcore-control")

response=acps.create_api_key_credential_provider(
    name="NasaInsightAPIKey", 
    apiKey="", # Get an API key by signing up at api.nasa.gov. Takes 2-min to get an API key in your email.
)

pprint(response)
credentialProviderARN = response['credentialProviderArn']
pprint(f"Egress Credentials provider ARN, {credentialProviderARN}")

ParamValidationError: Parameter validation failed:
Invalid length for parameter apiKey, value: 0, valid min length: 1

# Create an OpenAPI target 

#### Upload the NASA Open API json file in S3

In [ ]:
# Create an S3 client
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# Retrieve AWS account ID and region
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters
bucket_name = '' # Your s3 bucket to upload the OpenAPI json file.
file_path = 'openapi-specs/nasa_mars_insights_openapi.json'
object_key = 'nasa_mars_insights_openapi.json'
# Upload the file using put_object and read response
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # Construct the ARN of the uploaded object with account ID and region
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

#### Configure outbound auth and Create the gateway target

In [ ]:
# S3 Uri for OpenAPI spec file
nasa_openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

# API Key credentials provider configuration
api_key_credential_config = [
    {
        "credentialProviderType" : "API_KEY", 
        "credentialProvider": {
            "apiKeyCredentialProvider": {
                    "credentialParameterName": "api_key", # Replace this with the name of the api key name expected by the respective API provider. For passing token in the header, use "Authorization"
                    "providerArn": credentialProviderARN,
                    "credentialLocation":"QUERY_PARAMETER", # Location of api key. Possible values are "HEADER" and "QUERY_PARAMETER".
                    #"credentialPrefix": " " # Prefix for the token. Valid values are "Basic". Applies only for tokens.
            }
        }
    }
  ]

targetname='DemoOpenAPITargetS3NasaMars'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='OpenAPI Target with S3Uri using SDK',
    targetConfiguration=nasa_openapi_s3_target_config,
    credentialProviderConfigurations=api_key_credential_config)

# Calling Bedrock AgentCore Gateway from a Strands Agent

The Strands agent seamlessly integrates with AWS tools through the Bedrock AgentCore Gateway, which implements the Model Context Protocol (MCP) specification. This integration enables secure, standardized communication between AI agents and AWS services.

At its core, the Bedrock AgentCore Gateway serves as a protocol-compliant Gateway that exposes fundamental MCP APIs: ListTools and InvokeTools. These APIs allow any MCP-compliant client or SDK to discover and interact with available tools in a secure, standardized way. When the Strands agent needs to access AWS services, it communicates with the Gateway using these MCP-standardized endpoints.

The Gateway's implementation adheres strictly to the (MCP Authorization specification)[https://modelcontextprotocol.org/specification/draft/basic/authorization], ensuring robust security and access control. This means that every tool invocation by the Strands agent goes through authorization step, maintaining security while enabling powerful functionality.

For example, when the Strands agent needs to access MCP tools, it first calls ListTools to discover available tools, then uses InvokeTools to execute specific actions. The Gateway handles all the necessary security validations, protocol translations, and service interactions, making the entire process seamless and secure.

This architectural approach means that any client or SDK that implements the MCP specification can interact with AWS services through the Gateway, making it a versatile and future-proof solution for AI agent integrations.

In [ ]:
uv_install("mcp[cli]", "strands-agents")

# Request the access token from Amazon Cognito for inbound authorization

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# Ask Mars weather agent by calling NASA Open APIs using Bedrock AgentCore Gateway

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## The IAM group/user/ configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
    print(f"Tools loaded in the agent are {agent.tool_names}")
    print(f"Tools configuration in the agent are {agent.tool_config}")
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi , can you list all tools available to you")
    agent("What is the weather in northern part of the mars")
    # Invoke the agent with sample prompt, invoke the tool and display the response
    #Call the MCP tool explicitly. The MCP Tool name and arguments must match with your AWS Lambda function or the OpenAPI/Smithy API
    result = client.call_tool_sync(
    tool_use_id="get-insight-weather-1", # You can replace this with unique identifier. 
    name=targetname+"___getInsightWeather", # This is the tool name based on AWS Lambda target types. This will change based on the target name
    arguments={"ver": "1.0","feedtype": "json"}
    )
    #Print the MCP Tool response
    print(f"Tool Call result: {result['content'][0]['text']}")


**Issue: if you get below error while executing below cell, it indicates incompatibily between pydantic and pydantic-core versions.**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```
**How to resolve?**

You will need to make sure you have pydantic==2.7.2 and pydantic-core 2.27.2 that are both compatible. Restart the kernel once done.

# Clean up
Additional resources are also created like IAM role, IAM Policies, Credentials provider, AWS Lambda functions, Cognito user pools, s3 buckets that you might need to manually delete as part of the clean up. This depends on the example you run.

## Delete the gateway (Optional)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)